In [124]:
import pandas as pd

themes = {
    'A11_01': 'education',
    'A11_02': 'urban greenery',
    'A11_03': 'public space',
    'A11_04': 'welfare',
    'A11_05': 'culture',
    'A11_06': 'environmental protection',
    'A11_07': 'public transit and roads',
    'A11_08': 'sport',
    'A11_09': 'health'
}

# beneficiary_groups = {
#     'A12_01': 'families with children',
#     'A12_02': 'children',
#     'A12_03': 'youth',
#     'A12_04': 'adults',
#     'A12_05': 'people with disabilities',
#     'A12_06': 'elderly',
#     'A12_07': 'poor people'
# }

themes_mapping = {
    'education': 1, 'urban greenery': 2, 'public space': 3, 'welfare': 4, 'culture': 5,
    'environmental protection': 6, 'public transit and roads': 7, 'sport': 8, 'health': 9
}

# beneficiary_groups_mapping = {
#     'families with children': 1,
#     'children': 2,
#     'youth': 3,
#     'adults': 4,
#     'people with disabilities': 5,
#     'elderly': 6,
#     'poor people': 7
# }
#

education_mapping = {
    1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 3, 7: 3, 8: 3, 9: 3, 10: 4, 11: 4, 12: 4, 13: 4
}

marital_status_mapping = {
    'verheiratet': 1, 'in eingetragener Partnerschaft': 1, 'ledig': 2, 'geschieden': 2, 'verwitwet': 2
}

household_form_mapping = {
    'Einzelhaushalt': 1, 'Paar-/Zwei-Personenhaushalt': 2, 'Mehrpersonen-Haushalt ohne Kinder': 2, 'Mehrpersonen-Haushalt mit Kinder': 3
}

area_mapping = {
    'Altstadt': 1, 'Gönhard': 2, 'Zelgli': 3, 'Scheibenschachen': 4, 'Telli': 5,
    'Ausserfeld': 6, 'Innenstadt': 7, 'Hungerberg': 8, 'Goldern': 9, 'Binzenhof': 10
}

def find_positive_q1(row):
    positive_cols = []
    for col in q1_columns:
        value = pd.to_numeric(row[col], errors='coerce')
        if pd.notna(value) and value > 0:
            positive_cols.append(col.split('_')[-1])  # col.split('_')[-1] should be a string like '12', '13', etc.
    return positive_cols

def process_person(row):

    # max_beneficiary_score = max(row[key] for key in beneficiary_groups)
    # top_beneficiaries = [beneficiary_groups[key] for key in beneficiary_groups if row[key] == max_beneficiary_score]
    # top_beneficiary_codes = [str(beneficiary_groups_mapping.get(key, 0)) for key in top_beneficiaries]
    #
    # if max_beneficiary_score > 5:
    #     print(max_beneficiary_score)
    #     return None
    processed_data = {
        'ID': str(row['ID_Subjekt']),
        'Gender': '1' if row['S01_SEX'] == 1 else '2',
        'Age': row['S02'],  # Kept as integer
        'Politics': row['F03'],  # Kept as integer
        'Birthplace Info': '1' if row['K02'] == 0 or row['K03'] in [2, 3] else '2',
        'Education': str(education_mapping.get(row['K01'], 0)),
        'Nationality': '1' if row['Nationalitaet'] == 'Schweiz' else '2',
        'Marital Status': str(marital_status_mapping.get(row['Zivilstand'], 0)),
        'Household Form': str(household_form_mapping.get(row['Haushaltsform'], 0)),
        'Children Info': '1' if row['K04'] == 1 else '2' if row['K04'] == 2 else '0',
        'Area': str(area_mapping.get(row['Gebiet'], 0)),
        'Topics': [str(themes_mapping.get(themes[theme], 0)) for theme in themes if row[theme] == 1],
        'Votes': row['Votes']
    }

    return pd.Series(processed_data)

df1 = pd.read_csv('raw/w1_survey_data.csv')
df2 = pd.read_csv('raw/w2_survey_data.csv')

merged_df = pd.merge(df1, df2, left_on='IDNR', right_on='W1_IDNR')
merged_df['Votes'] = merged_df.apply(find_positive_q1, axis=1)

filtered_df = merged_df[merged_df['Q01_01'].notna()]
filtered_df = filtered_df[filtered_df['Votes'].astype(bool)]
filtered_df['F03'] = pd.to_numeric(filtered_df['F03'], errors='coerce')

filtered_df = filtered_df[filtered_df['F03'].between(0, 10, inclusive='both')]
processed_df = filtered_df.apply(process_person, axis=1).dropna()
processed_df = processed_df[processed_df['Topics'].astype(bool)]
print(processed_df.shape)

json_data = processed_df.to_json(orient='records')
with open('processed/aarau_pb_vote_processed.json', 'w', encoding='utf-8') as file:
    file.write(json_data)

processed_df.to_csv('processed/aarau_pb_vote_processed.csv', sep=',', encoding='utf-8', index=False)

(201, 13)


In [70]:
import pandas as pd

columns_to_keep = [
    'W1_IDNR', 'S00_SPRACHE', 'S01_SEX', 'S02', 'S02_AGEGRP', 'K00', 'F02_01', 'F02_02', 'F02_03', 'F02_04', 'F02_05',
    'F03', 'A11_01', 'A11_02', 'A11_03', 'A11_04', 'A11_05', 'A11_06', 'A11_07', 'A11_08', 'A11_09',
    'A12_01', 'A12_02', 'A12_03', 'A12_04', 'A12_05', 'A12_06', 'A12_07', 'K01', 'K02', 'K03', 'K04', 'K06',
    'PART', 'Q01_01', 'Q01_02', 'Q01_03', 'Q01_04', 'Q01_05', 'Q01_06', 'Q01_07', 'Q01_08', 'Q01_09', 'Q01_10',
    'Q01_11', 'Q01_12', 'Q01_13', 'Q01_14', 'Q01_15', 'Q01_16', 'Q01_17', 'Q01_18', 'Q01_19', 'Q01_20',
    'Q01_21', 'Q01_22', 'Q01_23', 'Q01_24', 'Q01_25', 'Q01_26', 'Q01_27', 'Q01_28', 'Q01_29', 'Q01_30',
    'Q01_31', 'Q01_32', 'Q01_33', 'Q01_WNKA', 'Geschlecht', 'Nationalitaet', 'Einwohnerstatus', 'Zivilstand', 'GEBMONATJAHR', 'AlterZP', 'Gebiet', 'Anzahl_Personen_Haushalt', 'Haushaltsform', 'ID_Subjekt'
]

q1_columns = [
    'Q01_01', 'Q01_02', 'Q01_03', 'Q01_04', 'Q01_05', 'Q01_06',
    'Q01_07', 'Q01_08', 'Q01_09', 'Q01_10', 'Q01_11', 'Q01_12',
    'Q01_13', 'Q01_14', 'Q01_15', 'Q01_16', 'Q01_17', 'Q01_18',
    'Q01_19', 'Q01_20', 'Q01_21', 'Q01_22', 'Q01_23', 'Q01_24',
    'Q01_25', 'Q01_26', 'Q01_27', 'Q01_28', 'Q01_29', 'Q01_30',
    'Q01_31', 'Q01_32', 'Q01_33'
]

w1_path = 'raw/w1_survey_data.csv'
w2_path = 'raw/w2_survey_data.csv'

df1 = pd.read_csv(w1_path)
df2 = pd.read_csv(w2_path)


merged_df = pd.merge(df1, df2, left_on='IDNR', right_on='W1_IDNR')

rows_in_w1 = df1.shape[0]
rows_in_w2 = df2.shape[0]
rows_in_combined = merged_df.shape[0]

filtered_df = merged_df[merged_df['Q01_01'].notna()]
filtered_df = filtered_df.loc[:, columns_to_keep]


def find_positive_q1(row):
    positive_cols = []
    for col in q1_columns:
        value = pd.to_numeric(row[col], errors='coerce')
        if pd.notna(value) and value > 0:
            positive_cols.append(col.split('_')[-1])
    return ', '.join(positive_cols)

filtered_df['Votes'] = filtered_df.apply(find_positive_q1, axis=1)
filtered_df = filtered_df[~filtered_df['Votes'].str.contains('#NULL!') & filtered_df['Votes'].str.strip().astype(bool)]

print(filtered_df)

     W1_IDNR  S00_SPRACHE  S01_SEX  S02 S02_AGEGRP  K00  F02_01  F02_02  \
1    1510541            1        2   36          2    1       3       3   
5    1510654            1        1   48          2    1       3       4   
12   1510782            1        2   47          2    1       3       3   
14   1510965            1        2   65          3    1       3       3   
19   1511022            1        1   30          1    1       3       3   
..       ...          ...      ...  ...        ...  ...     ...     ...   
766  1574627            1        2   68          3    1       4       4   
776  1576498            1        1   31          1    1       3       3   
779  1582128            1        1   71          3    1       4       4   
793  1584680            1        1   68          3    1       3       3   
801  1585834            1        1   74          3    1       4       4   

     F02_03  F02_04  ...  Nationalitaet   Einwohnerstatus   Zivilstand  \
1         3       3  ... 

In [68]:
def describe_person(row):

    education_levels = {
        1: 'basically educated',  # Combining No school, Primary, High school
        2: 'basically educated',
        3: 'basically educated',
        4: 'vocationally trained',  # Combining Apprenticeship, Vocational training
        5: 'vocationally trained',
        6: 'secondary-educated',  # General secondary, Commercial diploma, Advanced secondary diploma
        7: 'secondary-educated',
        8: 'secondary-educated',
        9: 'secondary-educated',
        10: 'highly educated',  # Advanced vocational training and University
        11: 'highly educated',
        12: 'highly educated',
        13: 'highly educated',
        98: 'unknown education level',
        99: 'no response on education'
    }

    beneficiary_groups = {
        'A12_01': 'families with children',
        'A12_02': 'children',
        'A12_03': 'youth',
        'A12_04': 'adults',
        'A12_05': 'people with disabilities',
        'A12_06': 'elderly',
        'A12_07': 'poor people'
    }

    themes = {
        'A11_01': 'education',
        'A11_02': 'urban greenery',
        'A11_03': 'public space',
        'A11_04': 'welfare',
        'A11_05': 'culture',
        'A11_06': 'environmental protection',
        'A11_07': 'public transit and roads',
        'A11_08': 'sport',
        'A11_09': 'health'
    }

    marital_status_translations = {
        'verheiratet': 'married',
        'ledig': 'single',
        'in eingetragener Partnerschaft': 'in a registered partnership',
        'geschieden': 'divorced',
        'verwitwet': 'widowed'
    }

    household_form_translations = {
        'Einzelhaushalt': 'single-person household',
        'Paar-/Zwei-Personenhaushalt': 'couple/two-person household',
        'Mehrpersonen-Haushalt ohne Kinder': 'multi-person household',
        'Mehrpersonen-Haushalt mit Kinder': 'multi-person households with children'
    }

    nationality_translations = {
        'Schweiz': 'Switzerland',
        'Afghanistan': 'Afghanistan',
        'Iran': 'Iran',
        'Spanien': 'Spain',
        'Deutschland': 'Germany',
        'Bosnien und Herzegowina': 'Bosnia and Herzegovina',
        'Österreich': 'Austria',
        'Griechenland': 'Greece',
        'Niederlande': 'Netherlands',
        'Vereinigte Staaten': 'United States',
        'Serbien': 'Serbia',
        'China': 'China',
        'Australien': 'Australia',
        'Tschechien': 'Czech Republic',
        'Italien': 'Italy',
        'Ukraine': 'Ukraine',
        'Polen': 'Poland',
        'Russland': 'Russia',
        'Frankreich': 'France',
        'Indien': 'India',
        'Kamerun': 'Cameroon',
        'Irak': 'Iraq',
        'Tunesien': 'Tunisia',
        'Mexiko': 'Mexico',
        'Türkiye': 'Turkey',
        'Litauen': 'Lithuania',
        'Bangladesch': 'Bangladesh',
        'Kosovo': 'Kosovo',
        'Portugal': 'Portugal',
        'Ungarn': 'Hungary'
    }

    def get_age_group(age):
        if age < 18:
            return 'Under 18'
        elif 18 <= age <= 29:
            return '18-29'
        elif 30 <= age <= 44:
            return '30-44'
        elif 45 <= age <= 64:
            return '45-64'
        else:
            return '65+'

    row['Gender'] = "man" if row['S01_SEX'] == 1 else "woman"

    row['Age'] = row['S02']
    row['Age Group'] = get_age_group(row['Age'])

    if row['F03'] in [0, 1]:
        row['Politics'] = "very left"
    elif row['F03'] < 5:
        row['Politics'] = "left-leaning"
    elif row['F03'] == 5:
        row['Politics'] = "moderate"
    elif row['F03'] < 9:
        row['Politics'] = "right-leaning"
    else:
        row['Politics'] = "very right"

    top_preferences = [themes[theme] for theme in themes if row[theme] == 1]
    row['Top Preferences'] = ', '.join(top_preferences)
    row['Education'] = education_levels.get(row['K01'], 'Unknown')
    row['Household Form'] = household_form_translations.get(row['Haushaltsform'], 'Unknown household form')
    translated_nationality = nationality_translations.get(row['Nationalitaet'], row['Nationalitaet'])
    row['Nationality'] = translated_nationality
    row['Marital Status'] = marital_status_translations.get(row['Zivilstand'], 'Unknown marital status')
    row['Area'] = row['Gebiet']

    row['Birthplace Info'] = "Has migration background" if row['K02'] == 0 or row['K03'] in [2, 3] else "No migration background"

    row['Children Info'] = 'Has children' if row['K04'] == 1 else 'No children' if row['K04'] == 2 else 'Unknown children status'

    # important_beneficiaries = [beneficiary_groups[key] for key in beneficiary_groups if row[key] == 5]
    # row['Important Beneficiaries'] = ', '.join(important_beneficiaries) if important_beneficiaries else 'None'

    description_parts = [
        f"You are a {row['Age']}-year-old {row['Gender']} living in {row['Area']} in Aarau, and you are {row['Politics']} politically."
    ]

    household_part = f"You live in a {row.get('Household Form', 'household')}."
    if row.get('Children Info') not in ['Unknown children status', '']:
        household_part += " You have children." if 'Has children' in row.get('Children Info', '') else ""
    description_parts.append(household_part)

    if row.get('Nationality') and row['Nationality'] != 'Switzerland':
        description_parts.append(f"You are originally from {row['Nationality']}.")

    if row.get('Birthplace Info') == "Has migration background":
        description_parts.append("You have a migration background.")

    if row.get('Top Preferences'):
        description_parts.append(f"Your top urban project preferences are: {row['Top Preferences']}.")

    # if row.get('Important Beneficiaries') and row['Important Beneficiaries'] != 'None':
    #     description_parts.append(f"For you, issues related to {row['Important Beneficiaries']} are very important.")

    if row.get('Education') == "highly educated":
        description_parts.append(f"You are highly educated.")

    row['Description'] = ' '.join(description_parts)

    print(row['Description'])
    return row


filtered_df = filtered_df.apply(describe_person, axis=1)

print(filtered_df.head())

You are a 36-year-old woman living in Hinterdorf in Aarau, and you are left-leaning politically. You live in a multi-person households with children. You have children. Your top urban project preferences are: education, environmental protection, public transit and roads. You are highly educated.
You are a 48-year-old man living in Gönhard in Aarau, and you are moderate politically. You live in a multi-person households with children. You have children. Your top urban project preferences are: education, public space, public transit and roads. You are highly educated.
You are a 47-year-old woman living in Scheibenschachen in Aarau, and you are left-leaning politically. You live in a multi-person households with children. You have children. Your top urban project preferences are: education, urban greenery, environmental protection. You are highly educated.
You are a 65-year-old woman living in Telli in Aarau, and you are right-leaning politically. You live in a couple/two-person household

In [37]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

categorical_cols = ['S01_SEX', 'F03', 'K01', 'Zivilstand', 'Haushaltsform', 'Nationalitaet']

stats_df = pd.get_dummies(filtered_df, columns=categorical_cols)

mlb = MultiLabelBinarizer()
votes_mlb = mlb.fit_transform(filtered_df['votes'])
votes_df = pd.DataFrame(votes_mlb, columns=mlb.classes_)
# print(stats_df.head())
# print(votes_df.head())

stats_df = pd.concat([stats_df, votes_df], axis=1).drop(['votes'], axis=1)
print(stats_df)
# print(stats_df.head())


       W1_IDNR  S00_SPRACHE   S02 S02_AGEGRP  K00  F02_01  F02_02  F02_03  \
0    1510527.0          1.0  35.0          2  1.0     3.0     4.0     3.0   
1    1510541.0          1.0  36.0          2  1.0     3.0     3.0     3.0   
2    1510546.0          1.0  41.0          2  1.0     3.0     3.0     3.0   
4    1510595.0          1.0  23.0          1  1.0     3.0     3.0     3.0   
5    1510654.0          1.0  48.0          2  1.0     3.0     4.0     3.0   
..         ...          ...   ...        ...  ...     ...     ...     ...   
501        NaN          NaN   NaN        NaN  NaN     NaN     NaN     NaN   
502        NaN          NaN   NaN        NaN  NaN     NaN     NaN     NaN   
504        NaN          NaN   NaN        NaN  NaN     NaN     NaN     NaN   
507        NaN          NaN   NaN        NaN  NaN     NaN     NaN     NaN   
508        NaN          NaN   NaN        NaN  NaN     NaN     NaN     NaN   

     F02_04  F02_05  ...    0    1    2    3    4    5    6    7    8    9 

In [22]:
def analyze_group_preferences(df, political_group):
    filtered_group = df[df['Politics'].isin(political_group)]
    average_age = filtered_group['Age'].mean()

    top_preferences = filtered_group['Top Preferences'].str.get_dummies(sep=', ').sum()
    sorted_top_preferences = top_preferences.sort_values(ascending=False)

    important_beneficiaries = filtered_group['Important Beneficiaries'].str.get_dummies(sep=', ').sum()
    sorted_important_beneficiaries = important_beneficiaries.sort_values(ascending=False)

    return average_age, sorted_top_preferences, sorted_important_beneficiaries

left_group = ['very left', 'left-leaning']
right_group = ['very right', 'right-leaning']

left_age, left_preferences, left_beneficiaries = analyze_group_preferences(filtered_df, left_group)
right_age, right_preferences, right_beneficiaries = analyze_group_preferences(filtered_df, right_group)

def print_group_analysis(group_name, age, preferences, beneficiaries):
    print(f"{group_name} Group\n")
    print(f"Average Age: {age:.2f} years")
    print("\nTop Preferences:")
    for preference, count in preferences.items():
        print(f"  - {preference.capitalize()}: {count}")
    print("\nImportant Beneficiaries:")
    for beneficiary, count in beneficiaries.items():
        print(f"  - {beneficiary.capitalize()}: {count}")
    print("\n")

print_group_analysis("Left", left_age, left_preferences, left_beneficiaries)
print_group_analysis("Right", right_age, right_preferences, right_beneficiaries)


Left Group

Average Age: 46.68 years

Top Preferences:
  - Environmental protection: 176
  - Education: 161
  - Welfare: 133
  - Urban greenery: 101
  - Public transit and roads: 90
  - Public space: 89
  - Culture: 66
  - Health: 58
  - Sport: 29

Important Beneficiaries:
  - Poor people: 128
  - Children: 119
  - Youth: 113
  - None: 102
  - Families with children: 96
  - People with disabilities: 80
  - Elderly: 49
  - Adults: 32


Right Group

Average Age: 54.68 years

Top Preferences:
  - Education: 70
  - Public transit and roads: 63
  - Health: 53
  - Environmental protection: 44
  - Sport: 44
  - Urban greenery: 41
  - Public space: 35
  - Welfare: 32
  - Culture: 24

Important Beneficiaries:
  - None: 66
  - Children: 36
  - Families with children: 35
  - People with disabilities: 34
  - Youth: 32
  - Elderly: 30
  - Poor people: 27
  - Adults: 16




In [24]:
# Define the columns to be saved
columns_to_save = ['ID_Subjekt', 'Gender', 'Age', 'Politics', 'Top Preferences',
                   'Education', 'Birthplace Info', 'Children Info', 'Important Beneficiaries', 'Description', 'Nationalitaet','Zivilstand','Gebiet','Haushaltsform', 'votes'
                   ]

# Create a subset of the DataFrame with only the specified columns
subset_df = filtered_df[columns_to_save]

# Filter the subset to include only rows where 'votes' is not empty
subset_df = subset_df[subset_df['votes'].notna() & (subset_df['votes'] != '')]

# Display the head of the DataFrame to verify
print(subset_df.head())
print(subset_df.shape)

# Save the filtered subset DataFrame to a CSV file
subset_df.to_csv('processed/aarau_pb_vote_final.csv', index=False)



  ID_Subjekt Gender  Age      Politics  \
0     105568  woman   35  left-leaning   
2     109007  woman   41  left-leaning   
4     110099    man   23  left-leaning   
6     104232    man   55    very right   
9     177339    man   34     very left   

                                     Top Preferences           Education  \
0  education, urban greenery, environmental prote...  secondary-educated   
2                         education, welfare, health     highly educated   
4  education, environmental protection, public tr...  secondary-educated   
6  education, public space, environmental protection     highly educated   
9    urban greenery, environmental protection, sport     highly educated   

            Birthplace Info Children Info  \
0  Has migration background   No children   
2   No migration background  Has children   
4  Has migration background   No children   
6  Has migration background   No children   
9   No migration background   No children   

                   